In [ ]:
!pip install transformers

In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl
%pip install -U datasets

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

modelName = "yatharth97/llama-2-7b-chat-yatharth-v2"

bnbConfig = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    modelName,
    device_map = {"": 0},
    quantization_config=bnbConfig
)

tokenizer = AutoTokenizer.from_pretrained(modelName)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [ ]:
import torch
# device = 0 if torch.cuda.is_available() else -1  # GPU if available, otherwise CPU


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# pipe = pipeline("text-generation", model="yatharth97/llama-2-7b-chat-yatharth-v2",device=device)

In [ ]:
system_prompt = ("<<SYS>> As a highly intelligent assistant, your primary goal is to provide accurate, "
                 "relevant, and context-aware responses to user queries based on the provided information. "
                 "Ensure your answers are factual, free from bias, and avoid promoting violence, hate speech, "
                 "or any form of discrimination. Focus on assisting the user effectively and safely. <</SYS>>")

In [ ]:
question="I have uploaded 10k report for Apple Inc. for the year 2023."
query = f"<s>[INST] {system_prompt}\n\n{question} [/INST]"


In [ ]:
# response = pipe(query)
# print(response[0]['generated_text'])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
#model.to(device)
custom_pipeline = pipeline(task="text-generation", model=model, tokenizer=tokenizer)


In [ ]:
result = custom_pipeline(query, max_length=256)
print(result[0]['generated_text'])

<s>[INST] <<SYS>> As a highly intelligent assistant, your primary goal is to provide accurate, relevant, and context-aware responses to user queries based on the provided information. Ensure your answers are factual, free from bias, and avoid promoting violence, hate speech, or any form of discrimination. Focus on assisting the user effectively and safely. <</SYS>>

Heads up, the 10k report for The Boeing Company is uploaded and ready for analysis. [/INST] answer: </s>010001000The Boeing Company:10K:The Boeing Company:2023:10K:United Airlines Holdings, Inc.:2023:10K:Delta Air Lines, Inc.:2023:10K:Delta Air Lines, Inc.:2023:10K:Delta Air Lines, Inc.:2023:10K:Delta Air Lines, Inc.:2023:10K:Delta Air Lines, Inc.:2023:10K:Delta Air Lines, Inc.:2023:


In [ ]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [ ]:
inputs = tokenizer(query, return_tensors="pt").to(device)  # Ensure inputs are on the same device as the model

# Generate response
# Note: Adjust `max_length` and `num_return_sequences` as needed
output_sequences = model.generate(
    **inputs, # Example max length
    max_length = 200,
    num_return_sequences=1,
   # early_stopping=True,
    repetition_penalty=1.2,
    top_p=0.92,
    temperature = 0.9
    #do_sample=True# Number of sequences to generate
)

# Decode the generated sequences to text
generated_text = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

# Print the generated text
for text in generated_text:
    print(text)

[INST] <<SYS>> As a highly intelligent assistant, your primary goal is to provide accurate, relevant, and context-aware responses to user queries based on the provided information. Ensure your answers are factual, free from bias, and avoid promoting violence, hate speech, or any form of discrimination. Focus on assisting the user effectively and safely. <</SYS>>

I have uploaded 10k report for Apple Inc. for the year 2023. [/INST] document_upload:10K:Apple Inc.:2023:10k:Annual Report:Fiscal Year:2023:Apple Inc.:United States:Technology:Telecommunications Equipment:NYSE:AAPL:2023-02-06:2023-02-06:10


In [ ]:
model.generate?